In [98]:
from pprint import pprint
from python_get_resolve import GetResolve

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()

project.SetSetting('timelineResolutionWidth', "3840")
project.SetSetting('timelineResolutionHeight', "2160")
media_pool.CreateEmptyTimeline("new timeline")
width_after_setting_project_res = project.GetCurrentTimeline().GetSetting()["timelineResolutionWidth"]
height_after_setting_project_res = project.GetCurrentTimeline().GetSetting()["timelineResolutionHeight"]
pprint(width_after_setting_project_res)
pprint(height_after_setting_project_res)
# project.GetCurrentTimeline().GetName()
# if project.GetCurrentTimeline().SetSetting("timelineResolutionWidth", "3840"):
#     print("SetSetting successful")
# if project.GetCurrentTimeline().SetSetting("timelineResolutionHeight", "2160"):
#     print("SetSetting height successful")

'3840'
'2160'
'3840'
'2160'
